In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models # models 모듈 추가

# 필요한 라이브러리 및 모듈 임포트 완료

# ==== 1. 데이터셋 로딩 및 전처리 (Fashion MNIST 및 ImageNet 전처리 적용) ====
# ImageNet으로 사전 학습된 모델은 일반적으로 3채널 이미지(RGB)와 특정 크기(예: 224x224)를 요구합니다.
# Fashion MNIST는 28x28 크기의 1채널(흑백) 이미지이므로, ImageNet 모델 입력에 맞게 변환해야 합니다.
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)), # 이미지 크기를 224x224로 조정
        transforms.ToTensor(), # 이미지를 텐서로 변환하고 값의 범위를 [0, 1]로 조정
        # 1채널 이미지를 3번 복사하여 3채널 이미지로 만듭니다 (흑백 이미지를 RGB처럼 처리)
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
        # ImageNet 데이터셋으로 학습될 때 사용된 평균 및 표준편차로 정규화
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

# MNIST 데이터셋 대신 FashionMNIST 데이터셋 로드
train_dataset = datasets.FashionMNIST(
    root="./data", train=True, download=True, transform=transform
)
val_dataset = datasets.FashionMNIST(
    root="./data", train=False, download=True, transform=transform
)

# 배치 사이즈 설정 (전이 학습 시 메모리 효율을 위해 작은 배치 사이즈 사용 권장)
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)

# ==== 2. 사용할 디바이스 설정 (M1 맥 최적화) ====
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():  # MPS 지원 확인
    device = torch.device("mps")  # MPS 사용
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# ==== 3. ImageNet 기반 사전 학습된 모델 불러오기 (ResNet18 예시) ====
# torchvision.models에서 ResNet18 모델을 불러오고, weights='ResNet18_Weights.DEFAULT'를 통해 사전 학습된 가중치를 로드합니다.
model = models.resnet18(weights='ResNet18_Weights.DEFAULT') # 사전 학습된 ResNet18 모델 로드

# 전이 학습을 위해 모델의 마지막 Fully Connected (FC) 레이어를 수정합니다.
# 사전 학습된 ResNet18은 ImageNet의 1000개 클래스를 분류하도록 되어 있지만,
# Fashion MNIST는 10개 클래스이므로 마지막 FC 레이어의 출력 뉴런 수를 10으로 변경해야 합니다.
num_ftrs = model.fc.in_features # 원래 마지막 FC 레이어의 입력 특성 수 확인
model.fc = nn.Linear(num_ftrs, 10) # 출력 뉴런 수를 10으로 갖는 새로운 FC 레이어로 교체

# 일반적으로 전이 학습 시에는 사전 학습된 부분(대부분의 레이어)의 가중치는 고정하고,
# 새로 추가하거나 수정한 레이어만 학습시킵니다 (Feature Extraction 방식).
# 마지막 FC 레이어를 제외한 나머지 레이어의 requires_grad를 False로 설정하여 학습을 비활성화합니다.
for name, param in model.named_parameters():
    if 'fc' not in name: # 레이어 이름에 'fc'가 포함되지 않으면 (마지막 FC 레이어가 아니면)
        param.requires_grad = False # 해당 레이어의 학습 비활성화

# 모델을 선택된 디바이스로 이동
model = model.to(device)
print(f"모델{device}.")

# ==== 4. 손실 함수 및 옵티마이저 정의 ====
# 손실 함수는 동일하게 CrossEntropyLoss를 사용합니다.
criterion = nn.CrossEntropyLoss()
# 옵티마이저 설정. requires_grad=False로 설정된 파라미터는 자동으로 학습 대상에서 제외됩니다.
optimizer = optim.Adam(model.parameters(), lr=0.001) # 학습 가능한 파라미터들만 최적화 대상으로 설정
# 전이 학습은 비교적 적은 에폭으로도 효과를 볼 수 있습니다. 에폭 수를 5로 설정합니다.
num_epochs = 5

best_val_acc = 0
best_model_state_dict = None # 최적 검증 정확도를 달성한 모델 상태 저장 변수

# ==== 5. 모델 학습 및 검증 루프 시작 ====
# 학습 및 검증 과정은 기본적으로 동일합니다.
print("모델 학습 시작.")
for epoch in range(num_epochs):
    model.train() # 모델을 훈련 모드로 설정
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    avg_train_loss = running_loss / len(train_loader)

    # ===== 검증 시작 =====
    model.eval() # 모델을 평가 모드로 설정
    val_correct = 0
    val_total = 0
    val_loss = 0.0

    with torch.no_grad(): # 검증 시에는 그래디언트 계산 비활성화
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            v_loss = criterion(outputs, labels)
            val_loss += v_loss.item()

            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total
    avg_val_loss = val_loss / len(val_loader)

    # 검증 정확도가 최고 기록을 갱신한 경우 모델 상태 저장
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state_dict = model.state_dict()
        print(
            f"  --> 검증 정확도 최고 기록 갱신: {val_acc:.2f}%. 현재 모델 상태 저장."
        )

    print(
        f"Epoch [{epoch+1}/{num_epochs}], "
        f"훈련 손실: {avg_train_loss:.4f}, 훈련 정확도: {train_acc:.2f}%, "
        f"검증 손실: {avg_val_loss:.4f}, 검증 정확도: {val_acc:.2f}%"
    )

print("모델 학습 완료.")

# ==== 6. 학습 완료 후 최적 모델 불러오기 (선택 사항) ====
# 학습 과정 중 가장 성능이 좋았던 시점의 모델 상태를 로드합니다.
if best_model_state_dict is not None:
    # 모델 구조를 다시 정의하고 저장된 상태를 로드
    final_model = models.resnet18(weights=None) # weights=None으로 시작 (사전 학습 가중치 없이)
    num_ftrs_final = final_model.fc.in_features
    final_model.fc = nn.Linear(num_ftrs_final, 10) # 마지막 FC 레이어 동일하게 수정
    final_model.load_state_dict(best_model_state_dict) # 저장된 상태 로드
    final_model = final_model.to(device) # 디바이스로 이동

    print(f"가장 성능이 좋았던 모델 상태를 로드했습니다. 최고 검증 정확도: {best_val_acc:.2f}%")
    # final_model을 사용하여 최종 테스트 또는 예측을 수행할 수 있습니다.
    # 사용 시에는 final_model.eval()로 평가 모드를 설정하십시오.


Using device: mps
모델mps.
모델 학습 시작.
  --> 검증 정확도 최고 기록 갱신: 83.91%. 현재 모델 상태 저장.
Epoch [1/5], 훈련 손실: 0.5683, 훈련 정확도: 80.81%, 검증 손실: 0.4639, 검증 정확도: 83.91%
  --> 검증 정확도 최고 기록 갱신: 85.61%. 현재 모델 상태 저장.
Epoch [2/5], 훈련 손실: 0.4348, 훈련 정확도: 84.37%, 검증 손실: 0.4132, 검증 정확도: 85.61%
Epoch [3/5], 훈련 손실: 0.4149, 훈련 정확도: 85.22%, 검증 손실: 0.4427, 검증 정확도: 84.36%
  --> 검증 정확도 최고 기록 갱신: 86.65%. 현재 모델 상태 저장.
Epoch [4/5], 훈련 손실: 0.3989, 훈련 정확도: 85.66%, 검증 손실: 0.3898, 검증 정확도: 86.65%
Epoch [5/5], 훈련 손실: 0.3940, 훈련 정확도: 85.89%, 검증 손실: 0.4045, 검증 정확도: 85.76%
모델 학습 완료.
가장 성능이 좋았던 모델 상태를 로드했습니다. 최고 검증 정확도: 86.65%
